### 제 2유형 연습하기. 팁 예측하기 (회귀)

In [2]:
import pandas as pd 
import numpy as np 

# 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 데이터분리
from sklearn.model_selection import train_test_split

# 모델 선정
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# 모델 성능 평가 
# 회귀 관련 성능 평가 
from sklearn.metrics import r2_score, mean_squared_error

# 분류 관련 성능 평가
from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score,accuracy_score

In [3]:
import seaborn as sns
df = sns.load_dataset('tips')

x= df.drop(columns='tip')
y = df['tip']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state=2024)
x_train= pd.DataFrame(x_train.reset_index())
x_test= pd.DataFrame(x_test.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_train.rename(columns={'index':'cust_id'},inplace=True)
x_test.rename(columns={'index':'cust_id'},inplace=True)
y_train.columns=['cust_id','tip']

cust_id = x_test['cust_id'].copy()

#### 레스토랑의 tip 예측 문제
##### - 데이터의 결측치, 이상치, 변수에 대해 처리하고
##### - 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오

데이터셋 설명    
- total_bill(총 청구액) : 손님의 식사 총 청구액
- tip(팁) : 팁의 양
- sex(성별) : 손님의 성별
- smoker(흡연자) : 손님의 흡연 여부 ("Yes" 또는 "No")
- day(요일) : 식사가 이루어진 요일
- time(시간) : 점심 또는 저녁 중 언제 식사가 이루어졌는지
- size(인원 수) : 식사에 참석한 인원수

#### 01. 데이터 탐색(EDA)

In [9]:
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

            count        mean        std   min    25%     50%      75%     max
cust_id     195.0  122.394872  70.734606  2.00  61.50  123.00  183.500  243.00
total_bill  195.0   19.754564   8.699479  3.07  13.38   17.59   24.535   48.33
size        195.0    2.589744   0.977002  1.00   2.00    2.00    3.000    6.00
            count        mean        std   min    25%     50%     75%     max
cust_id      49.0  117.938776  70.579567  0.00  56.00  115.00  172.00  239.00
total_bill   49.0   19.910816   9.763248  7.25  13.28   18.24   22.23   50.81
size         49.0    2.489796   0.844651  1.00   2.00    2.00    3.00    6.00
         count        mean        std  min   25%     50%      75%    max
cust_id  195.0  122.394872  70.734606  2.0  61.5  123.00  183.500  243.0
tip      195.0    2.946615   1.330591  1.0   2.0    2.71    3.525    9.0


In [11]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     195 non-null    int64   
 1   total_bill  195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   time        195 non-null    category
 6   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(2)
memory usage: 6.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     49 non-null     int64   
 1   total_bill  49 non-null     float64 
 2   sex         49 non-null     category
 3   smoker      49 non-null     category
 4   day         49 non-null     category
 5   time        49 non-null     category
 6   size   

In [14]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id    0
tip        0
dtype: int64
